In [1]:
# --- Install dependencies
!pip install -q torch scikit-learn joblib imbalanced-learn

In [2]:
# --- Imports ---
import os
import re
import json
import time
import random
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import joblib

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

In [3]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Device: cuda


In [4]:
# Hyperparameters (tune as needed)
RNN_CONFIG = {
    "embedding_dim": 100,   # if you use GloVe, match dim (100)
    "hidden_dim": 128,
    "num_layers": 1,
    "bidirectional": True,
    "dropout": 0.3,
    "batch_size": 64,
    "epochs": 6,
    "lr": 1e-3,
    "max_vocab_size": 30000,
    "max_seq_len": 200
}

In [5]:
df = pd.read_parquet("/content/engineered_features.parquet")
print("Dataset loaded. Shape:", df.shape)

# Try to find label column automatically
possible_label_cols = ['is_fake', 'fake', 'label', 'target', 'y']
label_col = None
for c in possible_label_cols:
    if c in df.columns:
        label_col = c
        break

if label_col is None:
    # fallback: try to find any binary column with 0/1 values
    for c in df.columns:
        if pd.api.types.is_integer_dtype(df[c]) or pd.api.types.is_float_dtype(df[c]):
            vals = set(df[c].dropna().unique())
            if vals.issubset({0,1}) or vals.issubset({0.0,1.0}):
                label_col = c
                break

if label_col is None:
    raise ValueError("No label column found. Please provide a binary label column (e.g. 'is_fake' with 0/1).")

print("Using label column:", label_col)

# Text column (same as earlier)
text_col = "clean_text"
if text_col not in df.columns:
    # try other likely names
    for c in ['text', 'review', 'reviews.text', 'review_text']:
        if c in df.columns:
            text_col = c
            break
    else:
        raise ValueError("No text column found. Please ensure 'clean_text' or a similar column exists.")

print("Using text column:", text_col)

# Drop NaNs
df = df[[text_col, label_col]].dropna().reset_index(drop=True)
df[text_col] = df[text_col].astype(str)

# Simple binary labels (0/1)
y = df[label_col].astype(int).values
texts = df[text_col].tolist()
print("Examples:", len(texts), "labels distribution:", np.bincount(y))

Dataset loaded. Shape: (9480, 47)
Using label column: purchase_missing_flag
Using text column: clean_text
Examples: 9480 labels distribution: [3447 6033]


In [6]:
# Train / Val / Test split + SMOTE oversampling

from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer

train_texts, test_texts, train_y, test_y = train_test_split(
    texts, y, test_size=0.15, random_state=RANDOM_SEED, stratify=y)

train_texts, val_texts, train_y, val_y = train_test_split(
    train_texts, train_y, test_size=0.17647, random_state=RANDOM_SEED, stratify=train_y)
# 70/15/15 split

print(f"Before SMOTE -> Train class distribution: {np.bincount(train_y)}")

# Convert text into numeric representation (TF-IDF) for SMOTE
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(train_texts)

# Apply SMOTE to balance classes
smote = SMOTE(random_state=RANDOM_SEED, sampling_strategy='auto')
X_resampled, y_resampled = smote.fit_resample(X_tfidf, train_y)

# Recover balanced texts (approximate mapping)
# We’ll rebuild the balanced text dataset using the same indices
# Note: synthetic samples don't have direct text forms (SMOTE works in vector space),
# so we’ll just duplicate some real examples to balance size for token-based models.

majority_idx = np.where(train_y == np.bincount(train_y).argmax())[0]
minority_idx = np.where(train_y == 1 - np.bincount(train_y).argmax())[0]

# Compute how many samples were generated by SMOTE
minority_upsampled_count = sum(y_resampled == (1 - np.bincount(train_y).argmax()))
synthetic_needed = minority_upsampled_count - len(minority_idx)

# Randomly duplicate minority texts to approximate the same balance for token-based models
duplicated_texts = np.random.choice(np.array(train_texts)[minority_idx], size=synthetic_needed, replace=True)
train_texts_balanced = list(train_texts) + duplicated_texts.tolist()
train_y_balanced = np.concatenate([train_y, np.full(synthetic_needed, 1 - np.bincount(train_y).argmax())])

print(f"After SMOTE -> Balanced train size: {len(train_texts_balanced)} | class distribution: {np.bincount(train_y_balanced)}")

Before SMOTE -> Train class distribution: [2413 4223]
After SMOTE -> Balanced train size: 8446 | class distribution: [4223 4223]


In [7]:
# Tokenization (simple whitespace split)
def tokenize(text: str) -> List[str]:
    return text.split()

# Build vocabulary from training data
from collections import Counter, defaultdict
counter = Counter()
for t in train_texts:
    counter.update(tokenize(t))

# Keep most common tokens
max_vocab = RNN_CONFIG["max_vocab_size"]
most_common = counter.most_common(max_vocab - 2)  # reserve indices for PAD and UNK
itos = ['<PAD>', '<UNK>'] + [w for w, _ in most_common]
stoi = {w: i for i, w in enumerate(itos)}
vocab_size = len(itos)
print("Vocab size:", vocab_size)

def text_to_sequence(tokens: List[str], stoi: Dict[str,int], max_len: int) -> List[int]:
    seq = [stoi.get(t, stoi['<UNK>']) for t in tokens][:max_len]
    if len(seq) < max_len:
        seq = seq + [stoi['<PAD>']] * (max_len - len(seq))
    return seq

# Option: initialize embedding matrix with preloaded GloVe (if available)
glove_path = "/content/glove.6B.100d.txt"
use_glove = os.path.exists(glove_path) and RNN_CONFIG["embedding_dim"] == 100
embedding_matrix = None

if use_glove:
    print("Loading GloVe for embedding initialization (this may take a minute)...")
    glove_map = {}
    with open(glove_path, 'r', encoding='utf8') as f:
        for line in f:
            parts = line.split()
            if len(parts) < 101:
                continue
            word = parts[0]
            vec = np.array(parts[1:], dtype='float32')
            glove_map[word] = vec
    # build embedding matrix
    embedding_matrix = np.random.normal(scale=0.6, size=(vocab_size, RNN_CONFIG["embedding_dim"])).astype('float32')
    for i, word in enumerate(itos):
        if word in glove_map:
            embedding_matrix[i] = glove_map[word]
    print("Embedding matrix ready (with GloVe where available).")

Vocab size: 8403
Loading GloVe for embedding initialization (this may take a minute)...
Embedding matrix ready (with GloVe where available).


In [8]:
# Create PyTorch dataset for RNN
class RNNReviewDataset(Dataset):
    def __init__(self, texts, labels, stoi, max_len):
        self.texts = texts
        self.labels = labels
        self.stoi = stoi
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        tokens = tokenize(self.texts[idx])
        seq = text_to_sequence(tokens, self.stoi, self.max_len)
        return {
            "input_ids": torch.tensor(seq, dtype=torch.long),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset_rnn = RNNReviewDataset(train_texts, train_y, stoi, RNN_CONFIG["max_seq_len"])
val_dataset_rnn   = RNNReviewDataset(val_texts, val_y, stoi, RNN_CONFIG["max_seq_len"])
test_dataset_rnn  = RNNReviewDataset(test_texts, test_y, stoi, RNN_CONFIG["max_seq_len"])

train_loader = DataLoader(train_dataset_rnn, batch_size=RNN_CONFIG["batch_size"], shuffle=True)
val_loader   = DataLoader(val_dataset_rnn, batch_size=RNN_CONFIG["batch_size"], shuffle=False)
test_loader  = DataLoader(test_dataset_rnn, batch_size=RNN_CONFIG["batch_size"], shuffle=False)


In [9]:
# Define LSTM-based classifier
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers=1, bidirectional=True, dropout=0.3, embedding_matrix=None):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        if embedding_matrix is not None:
            # freeze or not - we keep as trainable
            self.embedding.weight.data.copy_(torch.tensor(embedding_matrix))
        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, bidirectional=bidirectional, dropout=dropout if num_layers>1 else 0.0)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), 2)  # binary: 2 classes

    def forward(self, input_ids):
        emb = self.embedding(input_ids)           # (batch, seq_len, emb_dim)
        out, (hn, cn) = self.lstm(emb)            # out: (batch, seq_len, hidden*dir)
        # use mean pooling over seq dimension
        pooled = out.mean(dim=1)
        x = self.dropout(pooled)
        logits = self.fc(x)
        return logits

In [10]:
# Instantiate model
model_rnn = LSTMClassifier(
    vocab_size=vocab_size,
    emb_dim=RNN_CONFIG["embedding_dim"],
    hidden_dim=RNN_CONFIG["hidden_dim"],
    num_layers=RNN_CONFIG["num_layers"],
    bidirectional=RNN_CONFIG["bidirectional"],
    dropout=RNN_CONFIG["dropout"],
    embedding_matrix=embedding_matrix
).to(DEVICE)

# Training utilities
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_rnn.parameters(), lr=RNN_CONFIG["lr"])

In [11]:
def train_epoch_rnn(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    preds = []
    labels = []
    for batch in loader:
        input_ids = batch['input_ids'].to(device)
        lbls = batch['labels'].to(device)
        optimizer.zero_grad()
        logits = model(input_ids)
        loss = criterion(logits, lbls)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * input_ids.size(0)
        batch_preds = logits.argmax(dim=1).detach().cpu().numpy()
        preds.extend(batch_preds.tolist())
        labels.extend(lbls.detach().cpu().numpy().tolist())
    avg_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(labels, preds)
    return avg_loss, acc

In [12]:
def eval_rnn(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    preds = []
    labels = []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            lbls = batch['labels'].to(device)
            logits = model(input_ids)
            loss = criterion(logits, lbls)
            total_loss += float(loss) * input_ids.size(0)
            batch_preds = logits.argmax(dim=1).cpu().numpy()
            preds.extend(batch_preds.tolist())
            labels.extend(lbls.cpu().numpy().tolist())
    avg_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
    return avg_loss, acc, prec, rec, f1

In [13]:
# Train RNN
best_val_f1 = 0.0
print("\n--- Training RNN (LSTM) ---")
for epoch in range(1, RNN_CONFIG["epochs"] + 1):
    t0 = time.time()
    train_loss, train_acc = train_epoch_rnn(model_rnn, train_loader, optimizer, criterion, DEVICE)
    val_loss, val_acc, val_prec, val_rec, val_f1 = eval_rnn(model_rnn, val_loader, criterion, DEVICE)
    t1 = time.time()
    print(f"Epoch {epoch}/{RNN_CONFIG['epochs']} - train_loss {train_loss:.4f} train_acc {train_acc:.4f} | val_loss {val_loss:.4f} val_acc {val_acc:.4f} val_f1 {val_f1:.4f} time {t1-t0:.1f}s")
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model_rnn.state_dict(), "/content/best_rnn_model.pt")
        print("  -> Saved best_rnn_model.pt")


--- Training RNN (LSTM) ---


/tmp/ipython-input-956852357.py:14: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  total_loss += float(loss) * input_ids.size(0)


Epoch 1/6 - train_loss 0.6252 train_acc 0.6644 | val_loss 0.5694 val_acc 0.7018 val_f1 0.7909 time 3.3s
  -> Saved best_rnn_model.pt
Epoch 2/6 - train_loss 0.5746 train_acc 0.7030 | val_loss 0.5599 val_acc 0.7138 val_f1 0.8021 time 1.8s
  -> Saved best_rnn_model.pt
Epoch 3/6 - train_loss 0.4713 train_acc 0.7669 | val_loss 0.4421 val_acc 0.7862 val_f1 0.8302 time 2.1s
  -> Saved best_rnn_model.pt
Epoch 4/6 - train_loss 0.3797 train_acc 0.8228 | val_loss 0.4191 val_acc 0.8066 val_f1 0.8533 time 1.9s
  -> Saved best_rnn_model.pt
Epoch 5/6 - train_loss 0.3592 train_acc 0.8373 | val_loss 0.4072 val_acc 0.8193 val_f1 0.8696 time 2.2s
  -> Saved best_rnn_model.pt
Epoch 6/6 - train_loss 0.3369 train_acc 0.8492 | val_loss 0.4292 val_acc 0.7975 val_f1 0.8435 time 1.9s


In [14]:
# Evaluate on test set
print("\n--- RNN Test Evaluation ---")
model_rnn.load_state_dict(torch.load("/content/best_rnn_model.pt", map_location=DEVICE))
test_loss, test_acc, test_prec, test_rec, test_f1 = eval_rnn(model_rnn, test_loader, criterion, DEVICE)
print(f"RNN Test -> acc: {test_acc:.4f}, prec: {test_prec:.4f}, rec: {test_rec:.4f}, f1: {test_f1:.4f}")


--- RNN Test Evaluation ---
RNN Test -> acc: 0.8094, prec: 0.7952, rec: 0.9436, f1: 0.8631


In [15]:
# Save tokenizer & stoi/itos
joblib.dump({'itos': itos, 'stoi': stoi}, "/content/rnn_vocab.pkl")
print("RNN vocab saved to /content/rnn_vocab.pkl")

RNN vocab saved to /content/rnn_vocab.pkl
